# Phase 5: LangGraph Agent


## Step 0 — Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install langgraph langchain langchain-groq sentence-transformers faiss-cpu pandas numpy python-dotenv groq -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Load Environment + All Phase 2/3/4 Components

In [2]:
import os
import json
import re
import faiss
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from groq import Groq
from sentence_transformers import SentenceTransformer

load_dotenv()

DATA_DIR = Path('data')

# ── Load embedding model ───────────────────────────────────────
print('Loading embedding model...')
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# ── Load product FAISS index + metadata (Phase 2) ─────────────
product_index = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))
with open(DATA_DIR / 'faiss_metadata.json') as f:
    product_metadata = json.load(f)

# ── Load review FAISS index + metadata (Phase 4) ──────────────
review_index = faiss.read_index(str(DATA_DIR / 'reviews_faiss_index.bin'))
with open(DATA_DIR / 'reviews_metadata.json') as f:
    review_chunks = json.load(f)

# ── Load products dataframe ────────────────────────────────────
df_products = pd.read_csv(DATA_DIR / 'products_clean.csv')
median_price = df_products['price'].median()

# ── Groq client ───────────────────────────────────────────────
groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))
GROQ_MODEL  = 'llama-3.1-8b-instant'

print(f'✅ Product index  : {product_index.ntotal:,} vectors')
print(f'✅ Review index   : {review_index.ntotal:,} vectors')
print(f'✅ Products loaded: {len(product_metadata):,}')
print(f'✅ Review chunks  : {len(review_chunks):,}')
print(f'✅ Groq model     : {GROQ_MODEL}')

/Users/rithikabaskaran/MyFiles/ShopSense/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2308.79it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Product index  : 10,000 vectors
✅ Review index   : 3,008 vectors
✅ Products loaded: 10,000
✅ Review chunks  : 3,008
✅ Groq model     : llama-3.1-8b-instant


## Step 2 — Define the Three Agent Tools

Tools are just Python functions. The agent (LLM) decides which one to call.
Each tool does one specific job and returns a result the agent can use.

In [3]:
# ── TOOL 1: Search Tool ───────────────────────────────────────
# Searches products using FAISS semantic search.
# This is the main tool — called for almost every query.

def search_tool(query: str, top_k: int = 20) -> list[dict]:
    """
    Semantic product search using FAISS.
    Returns top_k most relevant products for the query.
    """
    query_emb = embed_model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    scores, indices = product_index.search(query_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        product = product_metadata[idx].copy()
        product['similarity_score'] = round(float(score), 4)
        # Fill NaN prices with median
        if not product.get('price') or str(product.get('price')) == 'nan':
            product['price'] = median_price
        results.append(product)

    return results


# ── TOOL 2: Filter Tool ───────────────────────────────────────
# Filters a list of products by price, rating, or keyword.
# Called AFTER search_tool when the user specifies constraints.

def filter_tool(
    products: list[dict],
    max_price: float = None,
    min_price: float = None,
    min_rating: float = None,
    keyword: str = None
) -> list[dict]:
    """
    Filter products by price range, minimum rating, or keyword.
    Always returns at least 3 results by relaxing filters if needed.
    """
    filtered = products

    if max_price is not None:
        filtered = [p for p in filtered if p.get('price', 0) <= max_price]
    if min_price is not None:
        filtered = [p for p in filtered if p.get('price', 0) >= min_price]
    if min_rating is not None:
        filtered = [p for p in filtered if p.get('rating', 0) >= min_rating]
    if keyword is not None:
        kw = keyword.lower()
        filtered = [
            p for p in filtered
            if kw in p.get('title', '').lower()
            or kw in p.get('description', '').lower()
        ]

    # Safety fallback — if too few results, relax filters
    if len(filtered) < 3:
        filtered = products[:10]

    return filtered


# ── TOOL 3: Clarify Tool ──────────────────────────────────────
# Used when the query is too vague to search meaningfully.
# Returns a follow-up question for the user instead of results.

def clarify_tool(query: str) -> str:
    """
    Generate a clarifying question when the user query is too vague.
    Returns a question string to ask the user.
    """
    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{
            'role': 'user',
            'content': f"""A user typed this shopping query: "{query}"

This query is too vague to search for products effectively.
Generate ONE short, friendly follow-up question to clarify what they need.
Focus on the most important missing detail (budget, use case, or specific type).
Keep it under 20 words. Just the question, no preamble."""
        }],
        max_tokens=50,
        temperature=0.5
    )
    return response.choices[0].message.content.strip()


print('✅ Three agent tools defined:')
print('   🔍 search_tool  — semantic product search')
print('   🔧 filter_tool  — price / rating / keyword filtering')
print('   ❓ clarify_tool — follow-up question for vague queries')

✅ Three agent tools defined:
   🔍 search_tool  — semantic product search
   🔧 filter_tool  — price / rating / keyword filtering
   ❓ clarify_tool — follow-up question for vague queries


## Step 3 — Define Agent State

The agent needs to remember things as it moves through its reasoning steps.
We store everything in a **state dictionary** that gets passed between nodes.
Think of it as the agent's short-term memory for one conversation turn.

In [4]:
from typing import TypedDict, Optional

class AgentState(TypedDict):
    """
    The agent's memory — passed between every node in the graph.

    Fields:
        query         : The original user message
        intent        : What the agent decided the user wants
                        ('search', 'filter', 'clarify')
        filters       : Any constraints extracted from the query
                        e.g. {'max_price': 30, 'min_rating': 4.0}
        search_results: Raw results from search_tool (top 20)
        final_results : Final results after filtering (top 5)
        clarify_question: Follow-up question if query was vague
        explanation   : Agent's explanation of its recommendations
        needs_clarification: True if agent wants to ask user something
    """
    query               : str
    intent              : Optional[str]
    filters             : Optional[dict]
    search_results      : Optional[list]
    final_results       : Optional[list]
    clarify_question    : Optional[str]
    explanation         : Optional[str]
    needs_clarification : bool

print('✅ AgentState defined.')
print('   Fields: query, intent, filters, search_results,')
print('           final_results, clarify_question, explanation, needs_clarification')

✅ AgentState defined.
   Fields: query, intent, filters, search_results,
           final_results, clarify_question, explanation, needs_clarification


## Step 4 — Define the Graph Nodes

Each node is a function that:
1. Receives the current state
2. Does one specific job
3. Returns an updated state

Our graph has 4 nodes:
```
┌─────────────┐     ┌──────────────┐     ┌─────────────┐     ┌─────────────┐
│   ANALYZE   │────▶│    SEARCH    │────▶│   FILTER    │────▶│   EXPLAIN   │
│  (intent +  │     │ (FAISS top  │     │ (apply user │     │ (generate   │
│   filters)  │     │    20)      │     │ constraints)│     │ explanation)│
└─────────────┘     └──────────────┘     └─────────────┘     └─────────────┘
       │
       │ (if vague)
       ▼
┌─────────────┐
│   CLARIFY   │
│ (ask user   │
│  question)  │
└─────────────┘
```

In [5]:
# ── NODE 1: Analyze ───────────────────────────────────────────
# The brain of the agent. Looks at the user's query and decides:
# - Is the query clear enough to search? Or too vague?
# - What filters did the user mention (price, rating)?
# - What is the user actually looking for?

def analyze_node(state: AgentState) -> AgentState:
    """
    Analyzes the user query to extract intent and filters.
    Decides whether to search or ask for clarification.
    """
    query = state['query']

    prompt = f"""You are a shopping assistant analyzing a user query.

User query: "{query}"

Analyze this query and respond with a JSON object containing:
{{
  "intent": "search" or "clarify",
  "needs_clarification": true or false,
  "search_query": "refined search query to use (improve if needed)",
  "filters": {{
    "max_price": null or number,
    "min_price": null or number,
    "min_rating": null or number,
    "keyword": null or string
  }},
  "reasoning": "one sentence explaining your decision"
}}

Rules:
- Set intent to "clarify" ONLY if the query is extremely vague (e.g. "something nice", "a good thing")
- Extract price from phrases like "under $30", "less than 50 dollars", "budget"
- Extract rating from phrases like "highly rated", "4 stars and above", "top rated"
- Improve the search_query to be more specific if needed
- Respond with valid JSON only"""

    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=200,
        temperature=0.1   # very low — we want consistent structured output
    )

    raw = response.choices[0].message.content.strip()

    try:
        clean   = re.sub(r'```json|```', '', raw).strip()
        parsed  = json.loads(clean)
        filters = parsed.get('filters', {})

        return {
            **state,
            'intent'             : parsed.get('intent', 'search'),
            'needs_clarification': parsed.get('needs_clarification', False),
            'query'              : parsed.get('search_query', query),
            'filters'            : {
                'max_price' : filters.get('max_price'),
                'min_price' : filters.get('min_price'),
                'min_rating': filters.get('min_rating'),
                'keyword'   : filters.get('keyword')
            }
        }
    except Exception:
        # If JSON parsing fails, default to search with no filters
        return {
            **state,
            'intent'             : 'search',
            'needs_clarification': False,
            'filters'            : {}
        }


# ── NODE 2: Clarify ───────────────────────────────────────────
# Only reached if analyze_node decided the query was too vague.
# Generates a follow-up question for the user.

def clarify_node(state: AgentState) -> AgentState:
    """Generates a clarifying question for vague queries."""
    question = clarify_tool(state['query'])
    return {
        **state,
        'clarify_question': question
    }


# ── NODE 3: Search ────────────────────────────────────────────
# Calls the FAISS search tool with the (possibly refined) query.
# Gets top 20 candidates.

def search_node(state: AgentState) -> AgentState:
    """Performs semantic search using FAISS."""
    results = search_tool(state['query'], top_k=20)
    return {
        **state,
        'search_results': results
    }


# ── NODE 4: Filter ────────────────────────────────────────────
# Applies any filters extracted by analyze_node.
# Narrows 20 candidates down to top 5.

def filter_node(state: AgentState) -> AgentState:
    """Applies price/rating/keyword filters and selects top 5."""
    filters  = state.get('filters') or {}
    products = state.get('search_results') or []

    filtered = filter_tool(
        products,
        max_price  = filters.get('max_price'),
        min_price  = filters.get('min_price'),
        min_rating = filters.get('min_rating'),
        keyword    = filters.get('keyword')
    )

    return {
        **state,
        'final_results': filtered[:5]
    }


# ── NODE 5: Explain ───────────────────────────────────────────
# The final node. Takes the top 5 results and generates a
# natural language explanation of WHY each product was recommended.

def explain_node(state: AgentState) -> AgentState:
    """Generates a natural language explanation for the recommendations."""
    query   = state['query']
    results = state.get('final_results') or []

    if not results:
        return {**state, 'explanation': 'No products found for your query.'}

    # Build a summary of the results for the LLM
    products_summary = '\n'.join([
        f"{i+1}. {r['title'][:60]} "
        f"(${r.get('price', 'N/A'):.2f} | {r.get('rating', 'N/A')}⭐)"
        for i, r in enumerate(results)
    ])

    prompt = f"""You are ShopSense, a helpful shopping assistant.

A user searched for: "{query}"

You recommended these products:
{products_summary}

Write a brief, friendly explanation (3-4 sentences) of:
1. Why these products match what the user is looking for
2. What makes the top recommendation stand out
3. Any useful shopping tip related to their query

Be conversational and helpful. No bullet points — just natural sentences."""

    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=150,
        temperature=0.7
    )

    return {
        **state,
        'explanation': response.choices[0].message.content.strip()
    }


print('✅ Five nodes defined:')
print('   1. analyze_node  — extract intent + filters')
print('   2. clarify_node  — ask follow-up if vague')
print('   3. search_node   — FAISS semantic search')
print('   4. filter_node   — apply constraints')
print('   5. explain_node  — generate explanation')

✅ Five nodes defined:
   1. analyze_node  — extract intent + filters
   2. clarify_node  — ask follow-up if vague
   3. search_node   — FAISS semantic search
   4. filter_node   — apply constraints
   5. explain_node  — generate explanation


## Step 5 — Build the LangGraph

Now we wire the nodes together into a graph.
The graph defines which node runs after which, and what decisions cause branching.

In [6]:
from langgraph.graph import StateGraph, END

# ── Create the graph with our state type ──────────────────────
graph = StateGraph(AgentState)

# ── Add all nodes to the graph ────────────────────────────────
graph.add_node('analyze', analyze_node)
graph.add_node('clarify', clarify_node)
graph.add_node('search',  search_node)
graph.add_node('filter',  filter_node)
graph.add_node('explain', explain_node)

# ── Set the entry point (first node to run) ───────────────────
graph.set_entry_point('analyze')

# ── Add conditional edge from analyze ─────────────────────────
# This is the branching logic:
# After analyze runs, check needs_clarification.
# If True  → go to clarify node (ask user a question)
# If False → go to search node (search for products)

def route_after_analyze(state: AgentState) -> str:
    """Routing function — decides which node to visit after analyze."""
    if state.get('needs_clarification'):
        return 'clarify'
    return 'search'

graph.add_conditional_edges(
    'analyze',                              # from this node
    route_after_analyze,                    # call this function to decide
    {'clarify': 'clarify', 'search': 'search'}  # map return value to node name
)

# ── Add straight edges (no branching) ─────────────────────────
# clarify → END (we stop and show the question to the user)
# search  → filter → explain → END
graph.add_edge('clarify', END)
graph.add_edge('search',  'filter')
graph.add_edge('filter',  'explain')
graph.add_edge('explain', END)

# ── Compile the graph into a runnable agent ───────────────────
agent = graph.compile()

print('✅ LangGraph agent compiled!')
print()
print('Graph flow:')
print('  START → analyze → [if vague] → clarify → END')
print('                  → [if clear] → search → filter → explain → END')

✅ LangGraph agent compiled!

Graph flow:
  START → analyze → [if vague] → clarify → END
                  → [if clear] → search → filter → explain → END


## Step 6 — Build the Agent Runner Function
A clean wrapper that runs the agent and displays results nicely.

In [7]:
def run_agent(query: str, verbose: bool = True) -> dict:
    """
    Run the ShopSense agent for a user query.

    Args:
        query   : Natural language user query
        verbose : If True, prints step-by-step agent reasoning

    Returns:
        Final agent state dict
    """
    # Initial state — everything empty except the query
    initial_state: AgentState = {
        'query'               : query,
        'intent'              : None,
        'filters'             : {},
        'search_results'      : None,
        'final_results'       : None,
        'clarify_question'    : None,
        'explanation'         : None,
        'needs_clarification' : False
    }

    print(f'\n{"="*65}')
    print(f'🛍️  ShopSense Agent')
    print(f'📝 Query: "{query}"')
    print(f'{"="*65}')

    # Run the graph — it handles all the node routing automatically
    final_state = agent.invoke(initial_state)

    # ── Display agent reasoning (verbose mode) ─────────────────
    if verbose:
        print(f'\n🧠 Agent Reasoning:')
        print(f'   Intent  : {final_state.get("intent", "search")}')
        filters = final_state.get('filters') or {}
        active_filters = {k: v for k, v in filters.items() if v is not None}
        if active_filters:
            print(f'   Filters : {active_filters}')
        else:
            print(f'   Filters : none')

    # ── Handle clarification case ──────────────────────────────
    if final_state.get('needs_clarification'):
        print(f'\n❓ Agent needs clarification:')
        print(f'   {final_state["clarify_question"]}')
        return final_state

    # ── Display results ────────────────────────────────────────
    results = final_state.get('final_results') or []
    print(f'\n📦 Top {len(results)} Recommendations:')
    print('─' * 65)

    for i, r in enumerate(results, 1):
        price_str = f"${r['price']:.2f}" if r.get('price') else 'N/A'
        print(f"{i}. {r['title'][:68]}")
        print(f"   💰 {price_str}  |  ⭐ {r.get('rating', 'N/A')} "
              f"({int(r.get('rating_count', 0)):,} reviews)")
        print(f"   Relevance score: {r.get('similarity_score', 'N/A')}")
        print()

    # ── Display explanation ────────────────────────────────────
    if final_state.get('explanation'):
        print('💬 Agent Explanation:')
        print(f"   {final_state['explanation']}")

    print(f'\n{"="*65}')
    return final_state


print('✅ run_agent() ready.')

✅ run_agent() ready.


## Step 7 — Test the Agent
Let's run several queries and watch the agent reason through each one.

In [8]:
# Test 1 — clear query, no filters
state1 = run_agent('compact coffee maker for small kitchen')


🛍️  ShopSense Agent
📝 Query: "compact coffee maker for small kitchen"

🧠 Agent Reasoning:
   Intent  : search
   Filters : none

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Small Coffee Table White Modern Wood Mini Rectangle Coffee Tables fo
   💰 $52.42  |  ⭐ 4.6 (80 reviews)
   Relevance score: 0.5822

2. Famiworths Single Serve Coffee Maker for K Cup and Ground Coffee, 6 
   💰 $22.99  |  ⭐ 3.9 (246 reviews)
   Relevance score: 0.537

3. Famiworths Single Serve Coffee Maker for K Cup and Ground Coffee, 6 
   💰 $46.18  |  ⭐ 4.3 (4,620 reviews)
   Relevance score: 0.5354

4. Black & Decker DCM7 Cup-at-a-Time Personal 12-Ounce Coffeemaker, Whi
   💰 $22.99  |  ⭐ 3.6 (237 reviews)
   Relevance score: 0.5084

5. Home Basics Stainless Steel Stove Top Espresso Coffee Maker (Silver)
   💰 $15.88  |  ⭐ 3.8 (52 reviews)
   Relevance score: 0.4936

💬 Agent Explanation:
   I think I've found some great options for you in compact coffee makers that 

In [9]:
# Test 2 — query with price filter (agent should extract max_price=30)
state2 = run_agent('non stick frying pan under $30')


🛍️  ShopSense Agent
📝 Query: "non stick frying pan under $30"

🧠 Agent Reasoning:
   Intent  : search
   Filters : {'max_price': 30}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Skillet Sma
   💰 $22.99  |  ⭐ 4.2 (29 reviews)
   Relevance score: 0.6875

2. MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan with Ce
   💰 $23.99  |  ⭐ 4.4 (10,637 reviews)
   Relevance score: 0.6424

3. NGORAY Pancake Egg Frying Pan Nonstick Mini - 4 Cups Cast Iron Omele
   💰 $22.99  |  ⭐ 4.1 (19 reviews)
   Relevance score: 0.6303

4. EOXHOME 8 Inch Non Stick Stone Coating Cookware Nonstick Skillet Fry
   💰 $22.99  |  ⭐ 4.5 (424 reviews)
   Relevance score: 0.6065

5. Copper Frying Pan 11-Inch Non Stick Ceramic Infused Titanium Steel O
   💰 $16.99  |  ⭐ 4.3 (57 reviews)
   Relevance score: 0.6061

💬 Agent Explanation:
   I've got just the thing for you if you're looking for a non-stick fryi

In [10]:
# Test 3 — query with rating filter
state3 = run_agent('highly rated storage bins for closet')


🛍️  ShopSense Agent
📝 Query: "highly rated storage bins for closet"

🧠 Agent Reasoning:
   Intent  : search
   Filters : {'min_rating': 4, 'keyword': 'storage bins'}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Plastic Storage Bins with Lids and Wheels - For Toys Shoes Clothes B
   💰 $22.99  |  ⭐ 4.6 (626 reviews)
   Relevance score: 0.6616

2. Superio Ribbed Collection - Decorative Plastic Lidded Home Storage B
   💰 $47.97  |  ⭐ 4.4 (36 reviews)
   Relevance score: 0.6502

3. Superio Ribbed Collection - Decorative Plastic Open Home Storage Bin
   💰 $29.83  |  ⭐ 4.4 (84 reviews)
   Relevance score: 0.6407

4. ANMINY Storage Bins with Zipper Lid Storage Boxes with Handles PP Pl
   💰 $26.99  |  ⭐ 4.7 (363 reviews)
   Relevance score: 0.6119

5. Foldable Storage Bins Linen Fabric with Lid and Handle for Home Bedr
   💰 $22.99  |  ⭐ 4.5 (20 reviews)
   Relevance score: 0.6036

💬 Agent Explanation:
   When searching for highly rated storage 

In [11]:
# Test 4 — vague query (agent should ask for clarification)
state4 = run_agent('something nice')


🛍️  ShopSense Agent
📝 Query: "something nice"

🧠 Agent Reasoning:
   Intent  : clarify
   Filters : none

❓ Agent needs clarification:
   What is this general product for (e.g. home, office, hobby, or a specific task)?


In [12]:
# Test 5 — query with both price and rating filters
state5 = run_agent('best knife set under $50 with 4 stars and above')


🛍️  ShopSense Agent
📝 Query: "best knife set under $50 with 4 stars and above"

🧠 Agent Reasoning:
   Intent  : search
   Filters : {'max_price': 50, 'min_rating': 4, 'keyword': 'knife set'}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. TOP CHEF - Dynasty 9-Piece Block Knife Set - German Stainless Steel 
   💰 $22.99  |  ⭐ 4.6 (17 reviews)
   Relevance score: 0.582

2. Kitchen Chef Knives Set with Wood Grain Handle 6 Pieces Stainless St
   💰 $22.99  |  ⭐ 4.4 (537 reviews)
   Relevance score: 0.5797

3. Daanaas Knife Set with Wooden Block and Sharpener 16 Pieces,Professi
   💰 $49.99  |  ⭐ 4.4 (54 reviews)
   Relevance score: 0.5569

4. Professional Chef Knife Set 5 Pieces, Kitchen Knives Set Stainless S
   💰 $22.99  |  ⭐ 4.3 (64 reviews)
   Relevance score: 0.5499

5. Cangshan S+ Series 1022094 German Steel Forged 2-Piece Titanium Coat
   💰 $49.95  |  ⭐ 4.7 (140 reviews)
   Relevance score: 0.5435

💬 Agent Explanation:
   You're looking f

In [13]:
# Test 6 — try your own query!
state6 = run_agent('gift for someone who loves cooking under $25')


🛍️  ShopSense Agent
📝 Query: "gift for someone who loves cooking under $25"

🧠 Agent Reasoning:
   Intent  : search
   Filters : {'max_price': 25}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. LITSTYLES Charcuterie Boards, Bamboo Cheese Board & Knife Set, Premi
   💰 $20.99  |  ⭐ 3.9 (26 reviews)
   Relevance score: 0.5198

2. Funny Fork Gifts for Women Men Kids, Cute My Mac and Cheese Fork Eng
   💰 $10.99  |  ⭐ 5.0 (11 reviews)
   Relevance score: 0.5071

3. Landisun Apron Kitchen Chef Cooking Gag Gift Creative Funny Grilling
   💰 $14.99  |  ⭐ 4.6 (154 reviews)
   Relevance score: 0.5005

4. Funny Gifts for Women Wife Girlfriend Friends, 20 oz Wine Tumbler wi
   💰 $22.99  |  ⭐ 4.6 (59 reviews)
   Relevance score: 0.4919

5. Not a Day Over Fabulous Wine Tumbler, Fun Birthday Gifts for Women W
   💰 $22.99  |  ⭐ 4.6 (354 reviews)
   Relevance score: 0.4717

💬 Agent Explanation:
   If you're looking for the perfect gift for someone who love

## Step 8 — Inspect Agent State
Shows exactly what the agent stored at each step — useful for debugging.

In [14]:
# Inspect the full state from Test 2 (price filter query)
print('Full agent state for "non stick frying pan under $30":')
print(f'  Query (refined) : {state2["query"]}')
print(f'  Intent          : {state2["intent"]}')
print(f'  Filters         : {state2["filters"]}')
print(f'  Results count   : {len(state2.get("final_results") or [])}')
print(f'  Explanation     : {state2.get("explanation", "")[:100]}...')

Full agent state for "non stick frying pan under $30":
  Query (refined) : non stick frying pan
  Intent          : search
  Filters         : {'max_price': 30, 'min_price': None, 'min_rating': None, 'keyword': None}
  Results count   : 5
  Explanation     : I've got just the thing for you if you're looking for a non-stick frying pan. These five products ma...


## Step 9 — Save Agent as Reusable Module
Saves `agent.py` so Phase 9 (FastAPI) can import it directly.

In [16]:
agent_module = '''
"""
ShopSense Agent Module
Reusable agent for Phase 9 FastAPI backend.
Usage:
    from agent import run_shopsense_agent
    result = run_shopsense_agent("compact coffee maker")
"""

import os, json, re, faiss, numpy as np, pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from groq import Groq
from sentence_transformers import SentenceTransformer
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

load_dotenv()
DATA_DIR   = Path("data")
GROQ_MODEL = "llama-3.1-8b-instant"

# Load all components
embed_model      = SentenceTransformer("all-MiniLM-L6-v2")
product_index    = faiss.read_index(str(DATA_DIR / "faiss_index.bin"))
review_index     = faiss.read_index(str(DATA_DIR / "reviews_faiss_index.bin"))
groq_client      = Groq(api_key=os.getenv("GROQ_API_KEY"))

with open(DATA_DIR / "faiss_metadata.json") as f:
    product_metadata = json.load(f)
with open(DATA_DIR / "reviews_metadata.json") as f:
    review_chunks = json.load(f)

df_products  = pd.read_csv(DATA_DIR / "products_clean.csv")
median_price = df_products["price"].median()


class AgentState(TypedDict):
    query: str
    intent: Optional[str]
    filters: Optional[dict]
    search_results: Optional[list]
    final_results: Optional[list]
    clarify_question: Optional[str]
    explanation: Optional[str]
    needs_clarification: bool


def search_tool(query, top_k=20):
    q = embed_model.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype("float32")
    scores, indices = product_index.search(q, top_k)
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1: continue
        p = product_metadata[idx].copy()
        p["similarity_score"] = round(float(score), 4)
        if not p.get("price") or str(p.get("price")) == "nan": p["price"] = median_price
        results.append(p)
    return results


def filter_tool(products, max_price=None, min_price=None, min_rating=None, keyword=None):
    f = products
    if max_price: f = [p for p in f if p.get("price", 0) <= max_price]
    if min_price: f = [p for p in f if p.get("price", 0) >= min_price]
    if min_rating: f = [p for p in f if p.get("rating", 0) >= min_rating]
    if keyword:
        kw = keyword.lower()
        f = [p for p in f if kw in p.get("title","").lower() or kw in p.get("description","").lower()]
    return f if len(f) >= 3 else products[:10]


def analyze_node(state):
    query = state["query"]
    prompt = f"""Analyze this shopping query: "{query}"
Respond with JSON: {{"intent": "search" or "clarify", "needs_clarification": bool,
"search_query": "improved query", "filters": {{"max_price": null or number,
"min_price": null or number, "min_rating": null or number, "keyword": null or string}}}}
Only clarify if extremely vague. JSON only."""
    r = groq_client.chat.completions.create(model=GROQ_MODEL,
        messages=[{"role": "user", "content": prompt}], max_tokens=200, temperature=0.1)
    try:
        p = json.loads(re.sub(r"```json|```", "", r.choices[0].message.content).strip())
        fi = p.get("filters", {})
        return {**state, "intent": p.get("intent", "search"),
                "needs_clarification": p.get("needs_clarification", False),
                "query": p.get("search_query", query),
                "filters": {"max_price": fi.get("max_price"), "min_price": fi.get("min_price"),
                             "min_rating": fi.get("min_rating"), "keyword": fi.get("keyword")}}
    except: return {**state, "intent": "search", "needs_clarification": False, "filters": {}}


def clarify_node(state):
    r = groq_client.chat.completions.create(model=GROQ_MODEL,
        messages=[{"role": "user", "content": f'Query: "{state["query"]}" is vague. Ask ONE clarifying question under 20 words.'}],
        max_tokens=50, temperature=0.5)
    return {**state, "clarify_question": r.choices[0].message.content.strip()}


def search_node(state):
    return {**state, "search_results": search_tool(state["query"])}


def filter_node(state):
    fi = state.get("filters") or {}
    filtered = filter_tool(state.get("search_results") or [],
        max_price=fi.get("max_price"), min_price=fi.get("min_price"),
        min_rating=fi.get("min_rating"), keyword=fi.get("keyword"))
    return {**state, "final_results": filtered[:5]}


def explain_node(state):
    results = state.get("final_results") or []
    if not results: return {**state, "explanation": "No products found."}
    summary = "\n".join([f"{i+1}. {r[\'title\'][:60]} (${r.get(\'price\',0):.2f} | {r.get(\'rating\')}⭐)"
                          for i, r in enumerate(results)])
    r = groq_client.chat.completions.create(model=GROQ_MODEL,
        messages=[{"role": "user", "content": f"User searched: \"{state[\'query\']}\"\nRecommended:\n{summary}\nWrite 3-4 friendly sentences explaining why these match."}],
        max_tokens=150, temperature=0.7)
    return {**state, "explanation": r.choices[0].message.content.strip()}


# Build graph
_graph = StateGraph(AgentState)
_graph.add_node("analyze", analyze_node)
_graph.add_node("clarify", clarify_node)
_graph.add_node("search",  search_node)
_graph.add_node("filter",  filter_node)
_graph.add_node("explain", explain_node)
_graph.set_entry_point("analyze")
_graph.add_conditional_edges("analyze",
    lambda s: "clarify" if s.get("needs_clarification") else "search",
    {"clarify": "clarify", "search": "search"})
_graph.add_edge("clarify", END)
_graph.add_edge("search",  "filter")
_graph.add_edge("filter",  "explain")
_graph.add_edge("explain", END)
_agent = _graph.compile()


def run_shopsense_agent(query: str) -> dict:
    """Main entry point for the ShopSense agent. Used by FastAPI in Phase 9."""
    state = _agent.invoke({
        "query": query, "intent": None, "filters": {},
        "search_results": None, "final_results": None,
        "clarify_question": None, "explanation": None,
        "needs_clarification": False
    })
    return {
        "query"              : state["query"],
        "needs_clarification": state.get("needs_clarification", False),
        "clarify_question"   : state.get("clarify_question"),
        "results"            : state.get("final_results") or [],
        "explanation"        : state.get("explanation"),
        "filters_applied"    : state.get("filters") or {}
    }
'''

with open('agent.py', 'w') as f:
    f.write(agent_module)

print('✅ agent.py saved — ready for Phase 9 FastAPI import')

✅ agent.py saved — ready for Phase 9 FastAPI import
